# RATING Project

Comic Mischief Detection Task

Files:

1. "train.csv" : 
-- Contains multiclass classification content annotations for each video scene used in the training set.
-- Annotations are on a scene level and do not correspond to a specific modality
-- a ".csv" file containing video URLs as well as the IDs of the scenes used in the training set.
-- Videos are available in the form of URLs, collected from the Youtube and the IMDB websites.
-- Contains metadata about the videos.
-- Four content categories related to comic mischief are used (Sarcasm, Slapstick Humor, Gory Humor, Mature Humor).

2. "val.csv" : 
-- Contains multiclass classification content annotations for each video scene used in the validation set.
-- You can use this set for performing model hyperparameter tuning before using the test set


3. "test.csv" : 
-- Contains multiclass classification content annotations for each video scene used in the test set.
-- You can use this set for evaluating your method

In [18]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [19]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

device_name = tf.test.gpu_device_name()
print("device name:", device_name)

tf.test.is_gpu_available()


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10397658043255654468
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6302793728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8382780069257635777
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:02:00.0, compute capability: 7.5"
]
2.6.0
Num GPUs Available:  1
device name: /device:GPU:0


True

In [20]:
tf.test.is_built_with_cuda()

True

In [21]:
tf.test.is_built_with_gpu_support()

True

In [22]:
# global variables

train_dir = os.getcwd() + "/train_data/"
val_dir = os.getcwd() + "/val_data/"
test_dir = os.getcwd() + "/test_data/"

# Hyperparameters
# MAX_SEQ_LENGTH = 1440

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 224

EPOCHS = 10
BATCH_SIZE = 64

### References
1. https://keras.io/examples/vision/video_transformers/)
2. https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub)
3. https://colab.research.google.com/github/sayakpaul/Action-Recognition-in-TensorFlow/blob/main/Data_Preparation_UCF101.ipynb

## Data Preparation (not complete)

### METADATA loading

In [23]:
# Create a dataframe which contains multiclass classification content annotations for each video scene used in the training set.
train_df = pd.read_csv('train-updated.csv', dtype={'combination': object}).iloc[:,1:]
train_df["path"] = train_dir + train_df["Video ID"]+ ".0" + train_df["Scene_ID"].astype(str) + ".mp4"
train_df.head()

,Video ID,Scene_ID,Video URL,Codec,Resolution,Avg Frame rate,Mature Humor - Scene Annotation,Slapstick Humor - Scene Annotation,Gory Humor - Scene Annotation,Sarcasm - Scene Annotation,combination,path
0,tt2872718,0,https://www.imdb.com/videoplayer/vi4179799833,h264,854 x 480,23.976024,0,0,0,0,0000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
1,tt2872718,1,https://www.imdb.com/videoplayer/vi4179799833,h264,854 x 480,23.976024,0,0,1,0,0010,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
2,tt2788710,0,https://www.imdb.com/videoplayer/vi1114222361,h264,854 x 480,23.976024,0,0,0,0,0000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
3,tt2788710,1,https://www.imdb.com/videoplayer/vi1114222361,h264,854 x 480,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
4,tt2788710,2,https://www.imdb.com/videoplayer/vi1114222361,h264,854 x 480,23.976024,0,0,0,0,0000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...


In [24]:
# Create a dataframe which contains multiclass classification content annotations for each video scene used in the validation set.
val_df = pd.read_csv('val.csv', dtype={'combination': object}).iloc[:,1:]
val_df["path"] = val_dir + val_df["Video ID"]+ ".0" + val_df["Scene_ID"].astype(str) + ".mp4"
val_df.head()

,Video ID,Scene_ID,Video URL,Codec,Resolution,Avg Frame rate,Mature Humor - Scene Annotation,Slapstick Humor - Scene Annotation,Gory Humor - Scene Annotation,Sarcasm - Scene Annotation,combination,path
0,tt1308728,0,https://www.youtube.com/watch?v=QP9qbhTeBII,h264,640 x 360,23.975945,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
1,tt1308728,1,https://www.youtube.com/watch?v=QP9qbhTeBII,h264,640 x 360,23.975945,1,1,0,0,1100,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
2,tt1308728,2,https://www.youtube.com/watch?v=QP9qbhTeBII,h264,640 x 360,23.975945,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
3,PGuqnE35cCg,0,https://www.youtube.com/watch?v=PGuqnE35cCg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
4,PGuqnE35cCg,1,https://www.youtube.com/watch?v=PGuqnE35cCg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...


In [25]:
# Create a dataframe which contains multiclass classification content annotations for each video scene used in the test set.
test_df = pd.read_csv('test-updated.csv', dtype={'combination': object}).iloc[:,1:]
test_df["path"] = test_dir + test_df["Video ID"]+ ".0" + test_df["Scene_ID"].astype(str) + ".mp4"
test_df.head()

,Video ID,Scene_ID,Video URL,Codec,Resolution,Avg Frame rate,Mature Humor - Scene Annotation,Slapstick Humor - Scene Annotation,Gory Humor - Scene Annotation,Sarcasm - Scene Annotation,combination,path
0,tt1741243,0,https://www.imdb.com/videoplayer/vi2169153049,h264,534 x 360,29.970000,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
1,tt1741243,1,https://www.imdb.com/videoplayer/vi2169153049,h264,534 x 360,29.970000,0,1,1,0,0110,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
2,tt1723121,0,https://www.youtube.com/watch?v=O7NHfAzg7Yg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
3,tt1723121,1,https://www.youtube.com/watch?v=O7NHfAzg7Yg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
4,tt1723121,2,https://www.youtube.com/watch?v=O7NHfAzg7Yg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...


### Data processing

How to feed videos to a neural network for training? <br>

1. Use OpenCV VideoCapture() method to read frames from videos.

In [26]:
# Utilities to open video files using CV2
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

#### Feature Extraction with pre-trained DenseNet121

In [27]:
def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

#### Label Preprocessing 

Multilabel classification



In [59]:
labels_data = []
for x in train_df["combination"].values:
    lst = list(map(int, x))
    arr = np.asarray(lst)
    # labels_data.append((list(map(int, x))))
    labels_data.append(arr)

# print(labels_data)
# label_processor = keras.layers.experimental.preprocessing.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(labels_data)
# )
# print(label_processor.get_vocabulary())


In [29]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["path"].values.tolist()
    # labels = df["combination"].values
    # labels = label_processor(labels[..., None]).numpy()


    labels = []
    for x in df["combination"].values:
        labels.append((list(map(int, x))))


    labels_data = []
    for x in train_df["combination"].values:
        lst = list(map(int, x))
        arr = np.asarray(lst)
        # labels_data.append((list(map(int, x))))
        labels_data.append(arr)


    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print(path)
        # Gather all its frames and add a batch dimension.
        frames = load_video(path)
        
        # Pad shorter videos.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate((frames, padding))

        frames = frames[None, ...]

        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features, labels

In [30]:
train_data, train_labels = prepare_all_videos(train_df, "train_data")
val_data, val_labels = prepare_all_videos(val_df, "val_data")
test_data, test_labels = prepare_all_videos(test_df, "test_data")

print(f"Frame features in train set: {train_data.shape}")
print(f"Train labels: {train_labels}")

c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2872718.00.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2872718.01.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2788710.00.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2788710.01.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2788710.02.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2243537.00.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt1980209.00.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt1980209.01.mp4
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt1980209

In [31]:
# Save to file
with open('train_data_2.npy', 'wb') as f:
    np.save(f, train_data)

with open('train_labels_2.npy', 'wb') as f:
    np.save(f, train_labels)

with open('val_data_2.npy', 'wb') as f:
    np.save(f, val_data)

with open('val_labels_2.npy', 'wb') as f:
    np.save(f, val_labels)

with open('test_data_2.npy', 'wb') as f:
    np.save(f, test_data)

with open('test_labels_2.npy', 'wb') as f:
    np.save(f, test_labels)

## Build the Transformer-based model (not completed) - BASE MODEL

In [32]:
# Embedding Layer
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [33]:
# Subclassed layer
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

### Utility functions for training

In [87]:
train_labels_data = []
for x in train_labels:
    arr = np.asarray(x)
    train_labels_data.append(arr)
train_labels_data = np.reshape(train_labels_data,(len(train_labels_data),4))


val_labels_data = []
for x in val_labels:
    arr = np.asarray(x)
    val_labels_data.append(arr)

val_labels_data = np.reshape(val_labels_data,(len(val_labels_data),4))


test_labels_data = []
for x in test_labels:
    arr = np.asarray(x)
    test_labels_data.append(arr)

test_labels_data = np.reshape(test_labels_data,(len(test_labels_data),4))


In [89]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    # classes = len(label_processor.get_vocabulary())
    classes = 4

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = os.getcwd() + "/tmp_2/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    with tf.device('/device:CPU:0'):
        model = get_compiled_model()
        history = model.fit(
            train_data,
            train_labels_data,
            validation_data=(val_data,val_labels_data),
            epochs=EPOCHS,
            callbacks=[checkpoint],
        )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels_data)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

## Model Training

In [ ]:
train_data, train_labels = np.load("train_data_2.npy"), np.load("train_labels_2.npy")
val_data, val_labels = np.load("val_data_2.npy"), np.load("val_labels_2.npy")
test_data, test_labels = np.load("test_data_2.npy"), np.load("test_labels_2.npy")

print(f"Frame features in train set: {train_data.shape}")

In [91]:
trained_model = run_experiment()

Epoch 1/10
30/30 [==============================] - 4s 93ms/step - loss: 0.8377 - accuracy: 0.3238 - val_loss: 0.5850 - val_accuracy: 0.2655

Epoch 00001: val_loss improved from inf to 0.58496, saving model to c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/tmp_2\video_classifier
Epoch 2/10
30/30 [==============================] - 3s 93ms/step - loss: 0.6173 - accuracy: 0.3142 - val_loss: 0.5614 - val_accuracy: 0.3097

Epoch 00002: val_loss improved from 0.58496 to 0.56137, saving model to c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/tmp_2\video_classifier
Epoch 3/10
30/30 [==============================] - 3s 92ms/step - loss: 0.5506 - accuracy: 0.3694 - val_loss: 0.5112 - val_accuracy: 0.3097

Epoch 00003: val_loss improved from 0.56137 to 0.51120, saving model to c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/tmp_2\video_classifier
Epoch 4/10
30/30 [=======================

In [ ]:
trained_model.summary()

## Evaluate the base model

In [99]:
def prepare_single_video(frames):
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # Pad shorter videos.
    if len(frames) < MAX_SEQ_LENGTH:
        diff = MAX_SEQ_LENGTH - len(frames)
        padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
        frames = np.concatenate(frames, padding)

    frames = frames[None, ...]

    # Extract features from the frames of the current video.
    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            if np.mean(batch[j, :]) > 0.0:
                frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
            else:
                frame_features[i, j, :] = 0.0

    return frame_features


# def predict_action(path):
#     class_vocab = label_processor.get_vocabulary()

#     frames = load_video(os.path.join("test", path))
#     frame_features = prepare_single_video(frames)
#     probabilities = trained_model.predict(frame_features)[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")

#     # pred = np.argsort(probabilities)[::-1]
#     # print(pred)
#     return frames


# # This utility is for visualization.
# # Referenced from:
# # https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


In [104]:
predict = []
actual = []

for path in test_df['path']:
    frames = load_video(os.path.join("test", path))
    frame_features = prepare_single_video(frames)
    y_pred = trained_model.predict(frame_features)[0]
    # round probabilities to class labels
    y_pred = y_pred.round()
    print(y_pred)
    predict.append(y_pred)

[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 1. 0.]
[1. 0. 0. 0.]
[1. 0. 1. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 1. 0.]
[1. 0. 1. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 1. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0.

In [107]:
test_labels_data

array([[1, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0,

In [106]:
recall = recall_score(y_true=test_labels_data, y_pred=predict, average='weighted')
print("Recall", recall)
precision = precision_score(y_true=test_labels_data, y_pred=predict, average='weighted')
print("Precision", precision)
f1 = f1_score(y_true=test_labels_data, y_pred=predict, average='weighted')
print("F1 score", f1)

Recall 0.5164835164835165
Precision 0.5202419202419202
F1 score 0.5122364477627636


C:\Users\maitr\anaconda3\envs\tf.2.6\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_video = np.random.choice(test_df["path"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = predict_action(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

# test_video_paths = test_df["path"].values.tolist()
# test_labels = test_df["combination"].values
# for idx, path in enumerate(test_video_paths):
#     print(f"Truth label: {test_labels[idx]}")
#     test_frames = predict_action(path)
#     print("\n")


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7327af42-8a03-4c46-b38e-e6931aa020f3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>